### 문제 1-3: 뉴스 키워드 추출기 (ChatPromptTemplate + FewShotPromptTemplate)

## 환경 설정

In [ ]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [1]:
# 환경 변수 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

True

In [2]:
# 필요한 라이브러리 import
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

## 해답 구현

### 1단계: 예시 데이터 준비

In [5]:
# 뉴스 키워드 추출 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라의 일론 머스크 CEO가 트위터를 인수한 후 대대적인 조직 개편을 단행했다. 직원 수를 대폭 줄이고 새로운 수익 모델을 도입하여 플랫폼의 지속가능성을 확보하려는 시도로 보인다.",
        "keywords": "테슬라, 일론머스크, 트위터"
    },
    {
        "news": "한국은행이 기준금리를 동결하기로 결정했다고 발표했다. 인플레이션 압력과 경제 성장률 둔화 사이에서 신중한 통화정책을 유지하겠다는 의지를 나타낸 것으로 분석된다.",
        "keywords": "한국은행, 기준금리, 인플레이션"
    }
]

### 2단계: ChatPromptTemplate을 사용한 예시 프롬프트 생성

In [6]:
# 대화형 예시 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

### 3단계: FewShotChatMessagePromptTemplate 구성

In [7]:
# FewShotChatMessagePromptTemplate 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

### 4단계: 최종 ChatPromptTemplate 구성

In [8]:
# 최종 ChatPromptTemplate
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 뉴스 키워드 추출 전문가입니다.
    
작업 지침:
1. 주어진 뉴스에서 가장 중요한 핵심 키워드 3개를 추출하세요
2. 고유명사(인명, 기업명, 기관명)를 우선적으로 고려하세요
3. 핵심 개념이나 기술 용어도 포함하세요
4. 결과는 반드시 "키워드: A, B, C" 형식으로 제시하세요
5. 키워드는 쉼표와 공백으로 구분하세요

다음 예시들을 참고하여 동일한 형식으로 답변하세요:"""),
    few_shot_prompt,
    ("human", "{input}")
])

### 5단계: 체인 구성 및 LLM 설정

In [9]:
# LLM 모델 초기화
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 출력 파서
output_parser = StrOutputParser()

# 체인 구성 (LCEL 사용)
chain = final_prompt | llm | output_parser

### 6단계: 테스트 실행

In [10]:
# 주어진 테스트 뉴스로 실행
test_news = """
제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.
"""

result = chain.invoke({"input": test_news})

print("뉴스 키워드 추출기 테스트 (ChatPromptTemplate 버전)")
print("=======================================================")
print("\n테스트 뉴스:")
print(test_news)
print("\n추출된 키워드:")
print(result)

뉴스 키워드 추출기 테스트 (ChatPromptTemplate 버전)

테스트 뉴스:

제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.


추출된 키워드:
키워드: 구글, 제미나이, API


### 7단계: 추가 테스트 케이스

In [11]:
# 다양한 분야의 추가 테스트
additional_tests = [
    {
        "category": "스포츠 뉴스",
        "text": "손흥민이 토트넘에서 프리미어리그 역대 아시아 선수 최다 득점 기록을 경신했다. 이로써 한국 축구의 위상이 한층 더 높아졌다는 평가를 받고 있다."
    },
    {
        "category": "경제 뉴스",
        "text": "비트코인 가격이 급등하면서 암호화폐 시장 전체가 활기를 띠고 있다. 투자자들의 관심이 다시 높아지고 있는 상황이다."
    },
    {
        "category": "환경 뉴스",
        "text": "정부가 탄소중립 목표 달성을 위해 재생에너지 확대 정책을 발표했다. 태양광과 풍력 발전 비중을 크게 늘리겠다고 밝혔다."
    },
    {
        "category": "기술 뉴스",
        "text": "애플이 차세대 아이폰에 새로운 AI 칩셋을 탑재할 예정이라고 발표했다. 이를 통해 온디바이스 AI 성능을 대폭 향상시킬 계획이다."
    }
]

print("추가 테스트 케이스들")
print("===================")

for i, test in enumerate(additional_tests, 1):
    result = chain.invoke({"input": test["text"]})
    print(f"\n테스트 {i} - {test['category']}:")
    print(f"뉴스: {test['text']}")
    print(f"결과: {result}")
    print("\n---")

추가 테스트 케이스들

테스트 1 - 스포츠 뉴스:
뉴스: 손흥민이 토트넘에서 프리미어리그 역대 아시아 선수 최다 득점 기록을 경신했다. 이로써 한국 축구의 위상이 한층 더 높아졌다는 평가를 받고 있다.
결과: 키워드: 손흥민, 토트넘, 프리미어리그

---

테스트 2 - 경제 뉴스:
뉴스: 비트코인 가격이 급등하면서 암호화폐 시장 전체가 활기를 띠고 있다. 투자자들의 관심이 다시 높아지고 있는 상황이다.
결과: 키워드: 비트코인, 암호화폐, 투자자

---

테스트 3 - 환경 뉴스:
뉴스: 정부가 탄소중립 목표 달성을 위해 재생에너지 확대 정책을 발표했다. 태양광과 풍력 발전 비중을 크게 늘리겠다고 밝혔다.
결과: 키워드: 탄소중립, 재생에너지, 태양광

---

테스트 4 - 기술 뉴스:
뉴스: 애플이 차세대 아이폰에 새로운 AI 칩셋을 탑재할 예정이라고 발표했다. 이를 통해 온디바이스 AI 성능을 대폭 향상시킬 계획이다.
결과: 키워드: 애플, 아이폰, AI칩셋

---


### 8단계: 생성된 프롬프트 구조 확인

In [10]:
# 실제 생성되는 프롬프트 구조 확인
sample_messages = final_prompt.format_messages(input="테스트 뉴스")

print("생성된 프롬프트 구조 확인")
print("==========================")
print(f"\n프롬프트 메시지 수: {len(sample_messages)}개")
print("\n메시지 구조:")
for i, message in enumerate(sample_messages, 1):
    content_preview = message.content[:50] + "..." if len(message.content) > 50 else message.content
    print(f"{i}. {message.__class__.__name__.lower()}: {content_preview}")

생성된 프롬프트 구조 확인

프롬프트 메시지 수: 10개

메시지 구조:
1. systemmessage: 당신은 뉴스 키워드 추출 전문가입니다.
    
작업 지침:
1. 주어진 뉴스에서 가장 중...
2. humanmessage: 삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다....
3. aimessage: 키워드: 삼성전자, 인공지능, 엔비디아
4. humanmessage: 세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했...
5. aimessage: 키워드: 세계보건기구, 건강위기, 국제협력
6. humanmessage: 테슬라의 일론 머스크 CEO가 트위터를 인수한 후 대대적인 조직 개편을 단행했다. 직원 수...
7. aimessage: 키워드: 테슬라, 일론머스크, 트위터
8. humanmessage: 한국은행이 기준금리를 동결하기로 결정했다고 발표했다. 인플레이션 압력과 경제 성장률 둔화 ...
9. aimessage: 키워드: 한국은행, 기준금리, 인플레이션
10. humanmessage: 테스트 뉴스


### 9단계: 성능 비교 및 분석

In [12]:
# 성능 분석
all_tests = [test_news] + [test["text"] for test in additional_tests]
results = []

for news in all_tests:
    result = chain.invoke({"input": news})
    # "키워드: " 부분 제거하고 키워드만 추출
    keywords_only = result.replace("키워드: ", "").split(", ")
    results.append(keywords_only)

# 통계 계산
total_tests = len(results)
successful_extractions = sum(1 for r in results if len(r) == 3)
avg_keywords = sum(len(r) for r in results) / len(results)

# 키워드 유형 분석 (간단한 휴리스틱)
proper_nouns = 0
tech_terms = 0
locations = 0

tech_keywords = ["AI", "인공지능", "암호화폐", "비트코인", "재생에너지", "탄소중립", "칩셋"]
location_keywords = ["WHO", "정부", "한국은행", "세계보건기구"]

for result in results:
    for keyword in result:
        if any(name in keyword for name in ["삼성", "테슬라", "손흥민", "애플", "구글", "일론"]):
            proper_nouns += 1
        if any(tech in keyword for tech in tech_keywords):
            tech_terms += 1
        if any(loc in keyword for loc in location_keywords):
            locations += 1

total_keywords = sum(len(r) for r in results)

print("성능 분석 결과")
print("==============")
print(f"\n총 테스트: {total_tests}개")
print(f"성공적인 키워드 추출: {successful_extractions}개 ({(successful_extractions/total_tests)*100:.1f}%)")
print(f"평균 키워드 개수: {avg_keywords:.1f}개")
print("\n키워드 유형 분석:")
print(f"- 고유명사 (기업/인명): {(proper_nouns/total_tests)*100:.1f}%")
print(f"- 기술/개념 용어: {(tech_terms/total_tests)*100:.1f}%")
print(f"- 지역/기관명: {(locations/total_tests)*100:.1f}%")
print("\n모든 테스트가 요구사항에 맞게 성공적으로 완료되었습니다!")

성능 분석 결과

총 테스트: 5개
성공적인 키워드 추출: 5개 (100.0%)
평균 키워드 개수: 3.0개

키워드 유형 분석:
- 고유명사 (기업/인명): 60.0%
- 기술/개념 용어: 120.0%
- 지역/기관명: 0.0%

모든 테스트가 요구사항에 맞게 성공적으로 완료되었습니다!
